# Load files from data

load all the articles from local storage (new_data & data) and prepare the tables for the subsequent analysis.

---

### Loading packages

In [1]:
import pandas as pd
import numpy as np
import glob
import re
import string
import datetime

## Loading data

The for-loop gathers all the .csv files within the data directory, concats and labels them and then returns a data frame.

In [2]:
category = {
    1: 'politik', 2: 'wirtschaft', 3: 'finanzen', 4: 'feuilleton', 5: 'sport', 6: 'gesellschaft', 7: 'stil', 
    8: 'technik-motor', 9: 'wissen', 10: 'reise', 11: 'beruf-chance'
}

In [3]:
def build_tables(category):
    raw_articles = []
    if category is 'aktuell':
        path = f"../new_data/aktuell/"
    else:
        path = f"../new_data/{category}/"
        
    all_files = glob.glob(path + '*.csv')

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        raw_articles.append(df)
    faz_articles = pd.concat(raw_articles, axis=0, ignore_index=True)
    faz_articles['label'] = category

    return faz_articles

---
---
# Generate Test and Train Data

**This for-loop goes through the data frames built in the first step and then collects the words from each entry in a list.**

## Train Data:

In [4]:
frames = []

for key, value in category.items():
    raw_faz = build_tables(value)
    frames.append(raw_faz)
    faz_train = pd.concat(frames, axis=0, ignore_index=True)

In [70]:
faz_train = faz_train.drop_duplicates()

In [1]:
#faz_train_seq = faz_train['detailed']
#faz_train_seq

In [6]:
#faz_train[faz_train['label'] == 'aktuell']

In [7]:
faz_train.groupby('label').count()

,link,published,title,detailed
label,,,,
beruf-chance,22,22,22,22
feuilleton,128,128,128,128
finanzen,84,84,84,84
gesellschaft,157,157,157,157
politik,292,292,292,292
reise,12,12,12,12
sport,268,268,268,268
stil,33,33,33,33
technik-motor,36,36,36,36


# Split train data in 'politik' and 'rest':

This step is for the sake of labelling the subsequant data and to get an idea of the data distribution (**'Politics' make up about 17% of the train data**):

In [8]:
faz_train.label.nunique()

11

In [9]:
faz_pol = faz_train[faz_train.label == 'politik']
faz_sport = faz_train[faz_train.label == 'sport']
faz_eco = faz_train[faz_train.label == 'wirtschaft']

In [10]:
faz_train_2 = faz_train.copy()

In [11]:
faz_train_2 = faz_train_2[faz_train_2.label != 'politik']
faz_train_2 = faz_train_2[faz_train_2.label != 'sport']
faz_train_2 = faz_train_2[faz_train_2.label != 'wirtschaft']
faz_rem = faz_train_2

In [12]:
faz_rem.label.nunique()

8

In [13]:
faz_rem.groupby('label').count()

,link,published,title,detailed
label,,,,
beruf-chance,22,22,22,22
feuilleton,128,128,128,128
finanzen,84,84,84,84
gesellschaft,157,157,157,157
reise,12,12,12,12
stil,33,33,33,33
technik-motor,36,36,36,36
wissen,51,51,51,51


---

# Build feature word list:

In [14]:
rgx_singles = re.compile("([\w][\w']*[\w])")
rgx_doubles = re.compile("([\w][\w']*[\w] +[\w][\w']*[\w])")
rgx_triples = re.compile("([\w][\w']*[\w] +[\w][\w']*[\w] +[\w][\w']*[\w])")

translator = str.maketrans('', '', string.punctuation)

In [15]:
words = []

for index, row in faz_train.iterrows():
    line = row['detailed'].translate(translator)
    words += rgx_singles.findall(line)
    words += rgx_doubles.findall(line)
    words += rgx_triples.findall(line)

In [16]:
word_list = pd.DataFrame(words, columns=['word'])

In [17]:
titles = []

for index, row in faz_train.iterrows():
    line = row['title'].translate(translator)
    titles += rgx_singles.findall(line)
    titles += rgx_doubles.findall(line)
    titles += rgx_triples.findall(line)

In [18]:
title_list = pd.DataFrame(titles, columns=['word'])

In [19]:
scraped = [word_list, title_list]
merged = pd.concat(scraped)

## Remove all words that are in the stopword list:

In [20]:
merged_list = merged['word'].tolist()

In [21]:
stopword = pd.read_csv('/Users/torben/PycharmProjects/toolbox/stopwords/stopwords.csv', index_col=None, header=0)
stopwordupper = pd.read_csv('/Users/torben/PycharmProjects/toolbox/stopwords/stopwordsupper.csv', index_col=None, header=0)

In [22]:
stopwords = stopword['words'].tolist()
stopwordsupper = stopwordupper['words'].tolist()

In [23]:
feature_list = [word for word in merged_list if word not in stopwords]
fin_feat_list = [word for word in feature_list if word not in stopwordsupper]

In [24]:
features = pd.DataFrame(fin_feat_list)

In [25]:
features.columns = ['words']

In [26]:
features['count'] = 1

In [27]:
features = features.groupby('words').count()

In [28]:
features = features.sort_values('count', ascending=False)
features_sort = features.head(1000)

In [29]:
feature_list = list(features_sort.index)
feature_list

['Jahre',
 'in der',
 'Trump',
 'Jahren',
 'Liveticker',
 'Zeit',
 'Trainer',
 'Assange',
 'deutschen',
 'Berlin',
 'Welt',
 'EU',
 'Deutschland',
 'Menschen',
 'Brexit',
 'für die',
 'Julian',
 'May',
 'deutsche',
 'zeigt',
 'Präsident',
 'Nowitzki',
 'Bayern',
 'Donald',
 'Spiel',
 'Jahr',
 'Polizei',
 'Bundesliga',
 'offenbar',
 'League',
 'Augsburg',
 'Dirk',
 'AfD',
 'Kinder',
 'lässt',
 'großen',
 'of',
 'FC',
 'Game',
 'München',
 'Thrones',
 'Theresa',
 'Game of',
 'Game of Thrones',
 'spricht',
 'britische',
 'große',
 'Eintracht',
 'London',
 'mit dem',
 'Champions',
 'Bild',
 'Amerika',
 'Europa',
 'an der',
 'lange',
 'Frankfurter',
 'China',
 'Wahl',
 'Deutsche',
 'Deutschen',
 'Euro',
 'Israel',
 'Verfolgen',
 'bei der',
 'gewinnt',
 'Merkel',
 'mit der',
 'Netanjahu',
 'Premierministerin',
 'Mittwoch',
 'stellt',
 'mit einem',
 'Kampf',
 'auf dem',
 'in die',
 'Woche',
 'Partei',
 'April',
 'Regierung',
 'Frankfurt',
 'Unternehmen',
 'auf die',
 'im Liveticker',
 'droht'

In [68]:
feature_list = pd.DataFrame(feature_list)
feature_list.to_csv('/Users/torben/PycharmProjects/toolbox/feature/features.csv', index=False)

In [30]:
dictionary={}

for i in range(len(feature_list)):
    dictionary[feature_list[i]] = i

---
---

# Iterate over 'politik' buzzwords:

In [31]:
pol_buzzword = []

for index, row in faz_pol.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    pol_buzzword.append(new_line)

In [32]:
df_pol = pd.DataFrame(pol_buzzword, columns=feature_list)

In [33]:
df_pol['goal_val'] = 1

# Iterate over 'sport' buzzwords:

In [34]:
sport_buzzword = []

for index, row in faz_sport.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    sport_buzzword.append(new_line)

In [35]:
df_sport = pd.DataFrame(sport_buzzword, columns=feature_list)

In [36]:
df_sport['goal_val'] = 2

# Iterate over 'wirtschaft' buzzwords:

In [37]:
eco_buzzword = []

for index, row in faz_eco.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    eco_buzzword.append(new_line)

In [38]:
df_eco = pd.DataFrame(eco_buzzword, columns=feature_list)

In [39]:
df_eco['goal_val'] = 3

# Iterate over remaining buzzwords:

In [40]:
rem_buzzword = []

for index, row in faz_rem.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    rem_buzzword.append(new_line)

In [41]:
df_rem = pd.DataFrame(rem_buzzword, columns=feature_list)

In [42]:
df_rem['goal_val'] = 0

In [43]:
tables = [df_pol, df_eco, df_sport, df_rem]
train_data = pd.concat(tables)

---
**saving the train data frames to a .csv file:**

In [44]:
name = 'train_data'

path = "../data_frames/" + name
print("Pathname:", path)

Pathname: ../data_frames/train_data


In [45]:
train_data.to_csv(path, index=False)

# SAME FOR TEST DATA:

In [46]:
def build_test_tables(category):
    raw_article = []
    if category is 'aktuell':
        path = f"../data/aktuell/"
    else:
        path = f"../data/{category}/"
        
    all_files = glob.glob(path + '*.csv')

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        raw_article.append(df)
    faz_article = pd.concat(raw_article, axis=0, ignore_index=True)
    faz_article['label'] = category

    return faz_article

In [47]:
fram = []

for key, value in category.items():
    raw_test = build_test_tables(value)
    fram.append(raw_test)
    faz_test = pd.concat(fram, axis=0, ignore_index=True)

In [48]:
faz_test = faz_test.drop_duplicates()

---

In [49]:
faz_pol_t = faz_test[faz_test.label == 'politik']
faz_sport_t = faz_test[faz_test.label == 'sport']
faz_eco_t = faz_test[faz_test.label == 'wirtschaft']

In [50]:
faz_test_2 = faz_test.copy()

In [51]:
faz_test_2 = faz_test_2[faz_test_2.label != 'politik']
faz_test_2 = faz_test_2[faz_test_2.label != 'sport']
faz_test_2 = faz_test_2[faz_test_2.label != 'wirtschaft']
faz_rem_t = faz_test_2

# pol_t:

In [52]:
pol_buzzword_t = []

for index, row in faz_pol_t.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    pol_buzzword_t.append(new_line)

In [53]:
df_pol_t = pd.DataFrame(pol_buzzword_t, columns=feature_list)

In [54]:
df_pol_t['goal_val'] = 1

# sport_t:

In [55]:
sport_buzzword_t = []

for index, row in faz_sport_t.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    sport_buzzword_t.append(new_line)

In [56]:
df_sport_t = pd.DataFrame(sport_buzzword_t, columns=feature_list)

In [57]:
df_sport_t['goal_val'] = 2

# eco_t:

In [58]:
eco_buzzword_t = []

for index, row in faz_eco_t.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    eco_buzzword_t.append(new_line)

In [59]:
df_eco_t = pd.DataFrame(eco_buzzword_t, columns=feature_list)

In [60]:
df_eco_t['goal_val'] = 3

# rem_t:

In [61]:
rem_buzzword_t = []

for index, row in faz_rem_t.iterrows():
    new_line = np.zeros(1000)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    rem_buzzword_t.append(new_line)

In [62]:
df_rem_t = pd.DataFrame(rem_buzzword_t, columns=feature_list)

In [63]:
df_rem_t['goal_val'] = 0

In [64]:
test_tables = [df_pol_t, df_sport_t, df_eco_t, df_rem_t]
top_test_data = pd.concat(test_tables)

---
**saving the data frames to a .csv file:**

In [65]:
name = 'test_data'

path = "../data_frames/" + name
print("Pathname:", path)

Pathname: ../data_frames/test_data


In [66]:
top_test_data.to_csv(path, index=False)

---